In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
                           
from idaes.core import FlowsheetBlock
from pyomo.opt import TerminationCondition, SolverStatus
                           
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.generic_models.unit_models import Flash

from idaes.core.util.model_statistics import degrees_of_freedom

import idaes.logger as idaeslog

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

In [2]:
from R125_bmimPF6_PR import configuration

In [37]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(-1.195988)
m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(-0.08814)
m.fs.properties.PR_kappa_B['bmimPF6', 'R125'].fix(1.56874)
m.fs.properties.PR_kappa_B['R125', 'bmimPF6'].fix(0.08489)

m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(283.15)
m.fs.F101.inlet.pressure.fix(699600)
m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(0.475)
m.fs.F101.inlet.mole_frac_comp[0,'bmimPF6'].fix(0.525)

# m.fs.F101.vap_outlet.pressure.fix(699600)
# m.fs.F101.vap_outlet.mole_frac_comp[0,'bmimPF6'] == 0.538
m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, 'bmimPF6'] == 0.538)
# m.fs.liquid2 = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, 'bmimPF6'] == 0.462)

m.fs.F101.vap_outlet.temperature.fix(283.15)

print(degrees_of_freedom(m.fs.F101))

1


In [38]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [39]:
m.fs.F101.initialize()    

# solve the model
status = solver.solve(m, tee = True)

2021-08-23 18:43:56 [INFO] idaes.init.fs.F101.control_volume.properties_in: Starting initialization
2021-08-23 18:43:56 [INFO] idaes.init.fs.F101.control_volume.properties_in: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-08-23 18:43:56 [INFO] idaes.init.fs.F101.control_volume.properties_in: Equilibrium temperature initialization completed.
2021-08-23 18:43:56 [INFO] idaes.init.fs.F101.control_volume.properties_in: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-08-23 18:43:57 [INFO] idaes.init.fs.F101.control_volume.properties_in: Property initialization: optimal - Optimal Solution Found.
2021-08-23 18:43:57 [INFO] idaes.init.fs.F101.control_volume.properties_out: Starting initialization
2021-08-23 18:43:57 [INFO] idaes.init.fs.F101.control_volume.properties_out: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-08-23 18:43:57 [INFO] idaes.init.fs.F101.control_volume.properties_out: Equilibrium temperatur

In [40]:
m.fs.F101.report()


Unit : fs.F101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value  : Fixed : Bounds
          Heat Duty : 93.832 : False : (None, None)
    Pressure Change : 4477.8 : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                              Inlet    Vapor Outlet  Liquid Outlet
    flow_mol                   1.0000     0.024164       0.97584  
    mole_frac_comp bmimPF6    0.52500   1.0000e-08       0.53800  
    mole_frac_comp R125       0.47500       1.0000       0.46200  
    temperature                283.15       283.15        283.15  
    pressure               6.9960e+05   7.0408e+05    7.0408e+05  


In [12]:
m.display()

Model unknown

  Variables:
    None

  Objectives:
    None

  Constraints:
    None

  Blocks:
    Block fs
    
      Variables:
        None
    
      Objectives:
        None
    
      Constraints:
        None
    
      Blocks:
        Block fs.properties
        
          Variables:
            PR_kappa_A : PRA binary interaction parameters
                Size=4, Index=fs.properties.PR_kappa_A_index
                Key                    : Lower : Value     : Upper : Fixed : Stale : Domain
                      ('R125', 'R125') :  None :       0.0 :  None :  True :  True :  Reals
                   ('R125', 'bmimPF6') :  None :  -0.08814 :  None :  True :  True :  Reals
                   ('bmimPF6', 'R125') :  None : -1.195988 :  None :  True :  True :  Reals
                ('bmimPF6', 'bmimPF6') :  None :       0.0 :  None :  True :  True :  Reals
            PR_kappa_B : PRB binary interaction parameters
                Size=4, Index=fs.properties.PR_kappa_B_index
     